# Load the dependcy

In [1]:
import math
import io
import torch
from torchvision import transforms
import numpy as np
import os

from PIL import Image, ImageChops

import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'



Map Encoder dependcy

In [2]:
import sys
import os
import clip
import torch
import cv2
import numpy as np
from PIL import Image
from  matplotlib import pyplot as plt
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.transforms import InterpolationMode
BICUBIC = InterpolationMode.BICUBIC
from segment_anything import sam_model_registry, SamPredictor

preprocess =  Compose([Resize((224, 224), interpolation=BICUBIC), ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))])
seg_model, preprocess = clip.load("CS-ViT-B/16", device=device)
seg_model.eval()

CLIPSurgery(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
   

Decoder dependcy

In [3]:
from utils.image import auto_resize, pad
import einops
from utils.common import instantiate_from_config, load_state_dict
from omegaconf import OmegaConf
from model.cldm import ControlLDM
from model.spaced_sampler import SpacedSampler
model: ControlLDM = instantiate_from_config(OmegaConf.load('./config/cldm.yaml'))
ckpt_swinir='./weights/general_full_v1.ckpt'
ckpt_net='./weights/cheng_small.pth.tar'
load_state_dict(model, torch.load(ckpt_swinir, map_location="cpu"), strict=True)
model.freeze()
model.to(device)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280,

D:\Anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


D:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: D:\Anaconda3\lib\site-packages\lpips\weights\v0.1\alex.pth


ControlLDM(
  (model): DiffusionWrapper(
    (diffusion_model): ControlledUnetModel(
      (time_embed): Sequential(
        (0): Linear(in_features=320, out_features=1280, bias=True)
        (1): SiLU()
        (2): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (input_blocks): ModuleList(
        (0): TimestepEmbedSequential(
          (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1-2): 2 x TimestepEmbedSequential(
          (0): ResBlock(
            (in_layers): Sequential(
              (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
              (1): SiLU()
              (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
            (h_upd): Identity()
            (x_upd): Identity()
            (emb_layers): Sequential(
              (0): SiLU()
              (1): Linear(in_features=1280, out_features=320, bias=True)
            )
            (out_layers): Sequential(
       

function

In [4]:
from skimage.transform import resize

import shutil
def clear_directory(directory_path):
    for file in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file)
        if os.path.isdir(file_path):
            # 如果是子文件夹，递归调用删除文件
            
            for root, dirs, files in os.walk(file_path):
                for file in files:
                    os.remove(os.path.join(root, file))
            
def to_block(img,grid=32,level=8):
    g_w=int(img.size[0]/grid)
    g_h=int(img.size[1]/grid)
    img_resize=img.resize((g_w, g_h))
    img_np=np.floor(np.array(img_resize)/level)*level
    img_np=img_np.astype (np.uint8)
    img_reference = Image.fromarray(img_np).resize(img.size)
    return img_reference, g_w, g_h

def divide_integer(num, n):
    quotient = num // n  # 整数除法，计算商
    remainder = num % n  # 取余数
    result = [quotient] * n  # 创建一个包含n个quotient的列表

    # 将余数平均分配给前几个数
    for i in range(remainder):
        result[i] += 1

    return result
def mask_block(mask,num=8,level=0.35):
    tmp=resize(mask, (num, num), mode='reflect')
    tmp[tmp>level]=255
    tmp[tmp<=level]=0
    rp_mat_0=np.array(divide_integer(mask.shape[0], num),dtype='int')
    rp_mat_1=np.array(divide_integer(mask.shape[1], num),dtype='int')
    return tmp.repeat(rp_mat_1,axis=1).repeat(rp_mat_0,axis=0)

def image_paddle_in(image, num=32):
    # 计算扩充后的宽度和高度
    new_width = ((image.width-1) // num + 1) * num
    new_height = ((image.height-1) // num + 1) * num

    # 创建一个新的扩充后的图像，用空值填充
    new_image = Image.new("RGB", (new_width, new_height), (0, 0, 0))

    # 将原始图像粘贴到扩充后的图像左上角
    new_image.paste(image, (0, 0))
    return new_image,image.width,image.height

def image_paddle_out(image, old_width, old_height):

    return image.crop((0,0,old_width,old_height))

def compute_bpp(out_net):
    size = out_net['x_hat'].size()
    num_pixels = size[0] * size[2] * size[3]
    return sum(torch.log(likelihoods).sum() / (-math.log(2) * num_pixels)
              for likelihoods in out_net['likelihoods'].values()).item()

def clip_map(img,texts,mask_num=8):
    image = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        # CLIP architecture surgery acts on the image encoder
        cv2_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        image_features = seg_model.encode_image(image)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)

        # Prompt ensemble for text features with normalization
        text_features = clip.encode_text_with_prompt_ensemble(seg_model, texts, device)

        # Extract redundant features from an empty string
        redundant_features = clip.encode_text_with_prompt_ensemble(seg_model, [""], device)

        # Apply feature surgery for single text
        similarity = clip.clip_feature_surgery(image_features, text_features, redundant_features)
        similarity_map = clip.get_similarity_map(similarity[:, 1:, :], cv2_img.shape[:2])
        
        mask_0=(similarity_map[0,:,:,0].cpu().numpy() * 255).astype('uint8')
        mask_1=(similarity_map[0,:,:,1].cpu().numpy() * 255).astype('uint8')
        mask_2=(similarity_map[0,:,:,2].cpu().numpy() * 255).astype('uint8')
        mask_0=Image.fromarray(mask_block(mask_0,num=mask_num))
        mask_1=Image.fromarray(mask_block(mask_1,num=mask_num))
        mask_2=Image.fromarray(mask_block(mask_2,num=mask_num))
        return mask_0,mask_1,mask_2
    
def sr_pipe(img_reference,positive_prompt="",cfg=1.0,steps=40,res=512):
    lq_resized = auto_resize(img_reference, res)
    x = pad(np.array(lq_resized), scale=64)
    sampler = SpacedSampler(model, var_type="fixed_small")
    control = torch.tensor(np.stack([x]) / 255.0, dtype=torch.float32, device=device).clamp_(0, 1)
    control = einops.rearrange(control, "n h w c -> n c h w").contiguous()
    control = model.preprocess_model(control)
    model.control_scales = [1] * 13
    height, width = control.size(-2), control.size(-1)
    shape = (1, 4, height // 8, width // 8)
    x_T = torch.randn(shape, device=device, dtype=torch.float32)
    samples = sampler.sample(
            steps=steps, shape=shape, cond_img=control,
            positive_prompt=positive_prompt, negative_prompt="Blurry, Low Quality", x_T=x_T,
            cfg_scale=cfg, cond_fn=None,
            color_fix_type="wavelet"
        )
    x_samples = samples.clamp(0, 1)
    x_samples = (einops.rearrange(x_samples, "b c h w -> b h w c") * 255).cpu().numpy().clip(0, 255).astype(np.uint8)

    img_sr=Image.fromarray(x_samples[0][:,0:lq_resized.size[0],:])
    return img_sr

# Encode and Decode Image

In [18]:
mode='ref'
using_map=False

if mode=='net':
    checkpoint_path=ckpt_net
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    state_dict = checkpoint
    for key in ["network", "state_dict", "model_state_dict"]:
        if key in checkpoint:
            state_dict = checkpoint[key]
    arch='cheng2020-attn'
    from compressai.zoo.image import model_architectures as architectures
    model_cls = architectures[arch]
    comp_net = model_cls.from_state_dict(state_dict).eval().to(device)
elif mode=='ref':
    ref_path='./ref/example-reference.png'
    ref_bpp=0.0421
elif mode=='pixel':
    block_level=3
    block_num_min=32

    
mask_num=8
res=1024

import pandas as pd
df = pd.read_csv("GPT-example.csv")

image_path='./example-input.png'


img = Image.open(image_path).convert('RGB')

########################## Encoder ##########################

#GPT prompt processing
prompt=df['prompt'][0]
prompt_list=prompt.split('\n')
prompt_list = [element for element in prompt_list if element != '']
name_0,name_1,name_2=prompt_list[0].split('.')[0].split(',')
detail_0,detail_1,detail_2=prompt_list[1],prompt_list[2],prompt_list[3]
detail_all=prompt_list[4]
mask_0,mask_1,mask_2=clip_map(img,[name_0,name_1,name_2],mask_num)

#reference
if mode=='pixel':
    old_width, old_height=img.size
    block_num=max(int(max(old_width, old_height)/16),block_num_min)
    img_reference=to_block(img,block_num,2**block_level)

    b_image=block_level*block_num**2
elif mode=='net':
    img, old_width, old_height = image_paddle_in(img, 64)
    x = transforms.ToTensor()(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out_net = comp_net.forward(x)
    out_net['x_hat'].clamp_(0, 1)
    img_reference = transforms.ToPILImage()(out_net['x_hat'].squeeze().cpu())
    img_reference = image_paddle_out(img_reference, old_width, old_height)
    b_image=compute_bpp(out_net)*img.size[0]*img.size[1]
elif mode=='ref':
    old_width, old_height=img.size
    img_reference = Image.open(ref_path).convert('RGB')
    b_image=ref_bpp*img.size[0]*img.size[1]


#############################################################

########################## Decoder ##########################
num_inference_steps=40
exag=1024/max(img_reference.size)
height=int(img_reference.size[1]*exag/8)*8
width=int(img_reference.size[0]*exag/8)*8

#    img_reference=img_reference.resize([width,height])
mask_0=mask_0.resize([width,height])
mask_1=mask_1.resize([width,height])
mask_2=mask_2.resize([width,height])
mask_all=Image.new("RGB", img_reference.size, (255, 255, 255))

image = img_reference 

if using_map:
    b_mask=mask_num*mask_num*3
    b_word=(len(detail_0)+len(detail_1)+len(detail_2)+len(detail_all))*8
    bpp=(b_image+b_mask+b_word)/(img.size[0]*img.size[1])
    print('bpp='+str(bpp))
    
    image_tmp = sr_pipe(image,positive_prompt=detail_0,cfg=3.5,steps=3,res=res)
    image = ImageChops.add(ImageChops.multiply(image_tmp,mask_0.convert("RGB")), 
                          ImageChops.multiply(image,Image.fromarray(255-np.array(mask_0)).convert("RGB"))
                          ).resize((old_width, old_height))
#    image.resize((old_width, old_height)).save(output_folder+'Mask0/'+image_name)

    image_tmp = sr_pipe(image,positive_prompt=detail_1,cfg=3.5,steps=3,res=res)
    image = ImageChops.add(ImageChops.multiply(image_tmp,mask_1.convert("RGB")), 
                          ImageChops.multiply(image,Image.fromarray(255-np.array(mask_1)).convert("RGB"))
                          ).resize((old_width, old_height))
#    image.resize((old_width, old_height)).save(output_folder+'Mask1/'+image_name)
                          
    image_tmp = sr_pipe(image,positive_prompt=detail_2,cfg=3.5,steps=3,res=res)
    image = ImageChops.add(ImageChops.multiply(image_tmp,mask_2.convert("RGB")), 
                          ImageChops.multiply(image,Image.fromarray(255-np.array(mask_2)).convert("RGB"))
                          ).resize((old_width, old_height))
#    image.resize((old_width, old_height)).save(output_folder+'Mask2/'+image_name)
    
    image = sr_pipe(image,positive_prompt=detail_all,cfg=7,steps=40,res=res).resize((old_width, old_height))
#    image.resize((old_width, old_height)).save(output_folder+'SR/'+image_name)
    
else:  
    b_word=(len(detail_all))*8
    bpp=(b_image+b_word)/(img.size[0]*img.size[1])
    print('bpp='+str(bpp))

    image = sr_pipe(image,positive_prompt=detail_all,cfg=7,steps=40,res=res)
    image = image.resize((old_width, old_height))
    



bpp=0.0469675537109375
timesteps used in spaced sampler: 
	[0, 26, 51, 77, 102, 128, 154, 179, 205, 231, 256, 282, 307, 333, 359, 384, 410, 435, 461, 487, 512, 538, 564, 589, 615, 640, 666, 692, 717, 743, 768, 794, 820, 845, 871, 897, 922, 948, 973, 999]


Spaced Sampler: 100%|██████████████████████████████████████████████████████████████████| 40/40 [00:17<00:00,  2.28it/s]
